[![preview notebook](https://img.shields.io/static/v1?label=render%20on&logo=github&color=87ce3e&message=GitHub)](https://github.com/open-atmos/PyMPDATA/blob/main/examples/PyMPDATA_examples/Olesik_et_al_2022/demo_make_convergences.ipynb)
[![launch on mybinder.org](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/open-atmos/PyMPDATA.git/main?urlpath=lab/tree/examples/PyMPDATA_examples/Olesik_et_al_2022/demo_make_convergences.ipynb)
[![launch on Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/open-atmos/PyMPDATA/blob/main/examples/PyMPDATA_examples/Olesik_et_al_2022/demo_make_convergences.ipynb)

see [Olesik et al. 2022](https://doi.org/10.5194/gmd-15-3879-2022)

In [1]:
import sys
if 'google.colab' in sys.modules:
    !pip --quiet install open-atmos-jupyter-utils
    from open_atmos_jupyter_utils import pip_install_on_colab
    pip_install_on_colab('PyMPDATA-examples')

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from open_atmos_jupyter_utils import show_plot
from joblib import Parallel, parallel_backend, delayed
from PyMPDATA.options import Options
from PyMPDATA_examples.Olesik_et_al_2022.settings import Settings, default_opt_set, option_string
from PyMPDATA_examples.Olesik_et_al_2022.coordinates import x_p2
from PyMPDATA_examples.Olesik_et_al_2022.simulation import Simulation
from PyMPDATA_examples.Olesik_et_al_2022.equilibrium_drop_growth import PdfEvolver
from PyMPDATA_examples.utils.discretisation import discretised_analytical_solution
from PyMPDATA_examples.utils.error_norms import Smolarkiewicz_Grabowski_1990_eq21

In [ ]:
CI = 'CI' in os.environ
GCs = np.linspace(.05,.95, 3) if CI else np.linspace(.05,.95, 7)
nrs = np.array([2**n for n in (range(7, 10) if CI else range(7,15))], dtype=int)

In [ ]:
grid_layout = x_p2()
psi_coord = x_p2()
settings = Settings()

In [ ]:
const = min(np.log2(1/nrs)) - 1


def phi(cour):
    return cour * np.pi / 2

def rho(n):
    return np.log2(1/n) - const


def polar_plot(nr, cour, values, name):
    theta_array = phi(cour)
    r_array = rho(nr)

    X, Y = np.meshgrid(theta_array, r_array)
    Z = np.array(list(values)).reshape(len(r_array), len(theta_array))

    min_val = -26
    max_val = -2

    amplitude = max_val - min_val
    if name == 'log$_2$(err)':
        levels = np.linspace(
            min_val,
            max_val,
            int(amplitude + 1)
        )
    else:
        levels = 7

    ax = plt.subplot(111, projection='polar')
    cnt = ax.contourf(X, Y, Z, levels, cmap='jet')
    plt.contour(X, Y, Z, levels, colors='black')
    ax.scatter(X, Y, alpha=.8, s=10)
    legend = plt.colorbar(cnt, ax=ax, pad=0.1)
    legend.set_label(r'$log_2(Err_{L2})$', rotation=90)
    ax.set_thetalim(min(theta_array),max(theta_array))
    ax.set_rlim(0, max(r_array))
    
    ticks = r_array
    ax.set_yticks(ticks)
    ax.set_yticklabels("$2^{" + f"{int(-tick - const):d}" + "}$" for tick in ticks)
    
    ax.set_thetagrids(cour * 90, tuple(f"{c:.2g}" for c in cour))
    ax.grid(True)
    ax.set_xlabel(r"$nx$", labelpad=18)
    ax.annotate(r'$C$', xy=(.8, .85), xycoords='axes fraction')

In [ ]:
def analysis(settings, GC, opt):
    options = Options(**opt)
    simulation = Simulation(settings, grid_layout,  psi_coord, options, GC)
    simulation.step(simulation.out_steps[-1])
    t = simulation.out_steps[-1] * simulation.dt
    rh = simulation.rh
    pdf_t = PdfEvolver(settings.pdf, settings.drdt, t)
    def pdf_arg(r):
        return pdf_t(r* rh.units).magnitude
    analytical = discretised_analytical_solution(
                simulation.rh.magnitude,
                pdf_arg, midpoint_value = True,
                r=simulation.r.magnitude
            ) * pdf_t(rh[0]).units
    numerical = simulation.n_of_r
    error = np.log2(Smolarkiewicz_Grabowski_1990_eq21(numerical.magnitude, analytical.magnitude, t.magnitude))
    return settings.nr, GC, error

In [ ]:
for opt in default_opt_set.values():
    with parallel_backend('threading', n_jobs=-2):
        results0 = Parallel(verbose=10)(
            delayed(analysis)(Settings(nr = nr, mixing_ratios_g_kg = [1.05,]), GC, opt)
            for nr in nrs
            for GC in GCs
        )
    results = tuple(tuple(i) for i in zip(*results0))
    plot_setup = np.array(results[0:2])
    measures = {'log$_2$(err)':results[2]}
    opt=option_string(str(opt))
    print(opt)
    for k, measure in measures.items():
        polar_plot(nrs, GCs, measure, name=k)
        show_plot(filename = f'convergence_{k}_{opt}.pdf')